In [ ]:
#!pip install qiskit
#!pip install qiskit[visualization]

In [ ]:
from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import Statevector
from qiskit.visualization import plot_histogram
from qiskit_aer import Aer
import numpy as np

n = 2
backend = Aer.get_backend('aer_simulator')

n_cube = pow(n,3)
norm = np.sqrt(n_cube)
nr_bits = int(np.floor(np.log2(n_cube)) + 1)
dims_value = int(pow(2,nr_bits))

coin_msgs = []
leader_msgs = []

### Send messages between processes
sender → sender_id 

receivers → set of processes meant to receive the message

circuit → message from sender to receiver(s)

In [ ]:
class Message:
    def __init__(self, sender, receivers, circuit) -> None:
        self.sender = sender
        self.receivers = receivers
        self.circuit = circuit
        self.result = None
        self.memory = None
    def __str__(self):
        return f"sender: {self.sender} | receivers: {self.receivers} | memory: {self.memory}"
    def measure_circuit(self):
        backend = Aer.get_backend('aer_simulator')
        #self.circuit = transpile(self.circuit, backend)
        self.circuit.save_statevector()
        self.result = backend.run(self.circuit, memory=True, shots=1).result()
        self.memory = self.result.get_memory()[0]

### Generate coin state

In [ ]:
def generate_coin(process_id):
    qc = QuantumCircuit(n)
    qc.h(0)
    for i in range(1,n):
        qc.cx(0,i)
        #qc.cx(i-1,i)
    qc.draw('mpl')
    qc.measure_all()
    new_msg = Message(process_id, list(range(0,n)), qc)
    coin_msgs.append(new_msg)

### Generate Leader state

In [ ]:
def tensor_n_qudits(qudit, nr_times):
    partial_copy = qudit.copy()
    tensor_vec = qudit.tensor(partial_copy)
    for i in range(1,nr_times-1):
            tensor_vec = tensor_vec.tensor(partial_copy)
    return tensor_vec

In [ ]:
def generate_leader(process_id):
    qudit = Statevector.from_int(1,dims=dims_value)
    final_ket = (1 / norm) * tensor_n_qudits(qudit,n)

    for i in range(2,n_cube+1):
        qudit = Statevector.from_int(i,dims=dims_value)
        final_ket += (1 / norm) * tensor_n_qudits(qudit,n)

    circuit_nr_bits = n * nr_bits
    another_qc = QuantumCircuit(circuit_nr_bits)
    another_qc.initialize(final_ket.data)
    another_qc.measure_all()

    new_msg = Message(process_id, list(range(0,n)), another_qc)
    leader_msgs.append(new_msg)

### QuantumCoinFlip

In [ ]:
def get_highest_leader():
    leader_measurements = []
    for i in range(len(leader_msgs)):
        process_id = leader_msgs[i].sender
        leader_outcome = leader_msgs[i].memory[:nr_bits]
        leader_measurements.append((process_id, leader_outcome))
    return max(leader_measurements, key=lambda x:x[1])

In [ ]:
for i in range(n):
    generate_coin(i)
    generate_leader(i)

for i in range(len(leader_msgs)):
    leader_msgs[i].measure_circuit()
    print(leader_msgs[i].memory)

leader_process_id, highest_leader = get_highest_leader()